# Customer Churn Prediction

Customer churn prediction is to measure why customers are leaving a business. In this tutorial we will be looking at customer churn in Bank business. We will build a deep learning model to predict the churn and use precision, recall, f1-score to measure performance of our model.

Disclaimer: You should perform under / over sample (including SMOTE) only on training data, and measure f1 on original data distribution (test data). Moreover, if you divide oversample data with train_test_split then you have no control over the distribution of duplicated items for test and train. Which means that you can have the same observation in both test and train, which means you test partially on the training set - that's why the results increase. So first divide into train / test, and then perform operations only on the training set, and the test set should be without any changes. 


In [82]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

LOAD THE DATA

In [83]:
df = pd.read_csv('data/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


DROP UNNECCESSARY COLUMNS

In [84]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


FEATURE ENGINEERING

In [85]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [86]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [87]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


ONE HOT ENCODING CATEGORICAL VALUES

In [88]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [89]:
print_unique_col_values(df1)

In [90]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


SCALING THE DATASET

In [91]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [92]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [93]:
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])

In [94]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


In [95]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

In [96]:
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [97]:
X.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [98]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

SPLITTING THE DATASET INTO TRAINING AND TEST SET

In [99]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)


In [100]:
X_train.shape

(8000, 13)

In [101]:
X_test.shape

(2000, 13)

IMPORTING TENSORFLOW LIBRARIES

In [102]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

BUILD THE MODEL(ANN)

In [103]:
def ANN(X_train, y_train, X_test, y_test, loss):
    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [104]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5021 - accuracy: 0.7872
Epoch 2/100
 41/250 [===>..........................] - ETA: 0s - loss: 0.4573 - accuracy: 0.8041

250/250 [==============================] - 1s 3ms/step - loss: 0.4698 - accuracy: 0.7991
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4587 - accuracy: 0.8066
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4465 - accuracy: 0.8064
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4337 - accuracy: 0.8111
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4225 - accuracy: 0.8175
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4143 - accuracy: 0.8180
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4040 - accuracy: 0.8253
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3946 - accuracy: 0.8288
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3849 - accuracy: 0.8350
Epoch 11/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3776 - accuracy: 0.8389


As We See, the precision, recall and f1 score of Class 1 is very low due to imbalanced dataset

## Mitigating Skewdness of Data

### Method 1: Undersampling

reference: https: // www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets


In [105]:
# Class count
count_class_0, count_class_1 = df1.Exited.value_counts()

# Divide by class
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [106]:
print(count_class_0)
print(count_class_1)

7963
2037


In [107]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Exited.value_counts())

Random under-sampling:
0    2037
1    2037
Name: Exited, dtype: int64


In [108]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [109]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [110]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100


102/102 [==============================] - 1s 3ms/step - loss: 0.7054 - accuracy: 0.5149
Epoch 2/100
102/102 [==============================] - 0s 3ms/step - loss: 0.6651 - accuracy: 0.6158
Epoch 3/100
102/102 [==============================] - 0s 3ms/step - loss: 0.6467 - accuracy: 0.6401
Epoch 4/100
102/102 [==============================] - 0s 3ms/step - loss: 0.6359 - accuracy: 0.6468
Epoch 5/100
102/102 [==============================] - 0s 3ms/step - loss: 0.6286 - accuracy: 0.6591
Epoch 6/100
102/102 [==============================] - 0s 3ms/step - loss: 0.6187 - accuracy: 0.6597
Epoch 7/100
102/102 [==============================] - 0s 4ms/step - loss: 0.6091 - accuracy: 0.6698
Epoch 8/100
102/102 [==============================] - 0s 3ms/step - loss: 0.5961 - accuracy: 0.6830
Epoch 9/100
102/102 [==============================] - 0s 3ms/step - loss: 0.5822 - accuracy: 0.6938
Epoch 10/100
102/102 [==============================] - 0s 4ms/step - loss: 0.5681 - accuracy: 0.7042
E

As we see, there is considerable Improvement in the f1, recall and precision scores of Class 1 Value.
The f1 score has improved from 0.58 to 0.77.

### Method 2: Oversampling

In [112]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Exited.value_counts())

Random over-sampling:
0    7963
1    7963
Name: Exited, dtype: int64


In [113]:
from sklearn.model_selection import train_test_split
X = df_test_over.drop('Exited', axis='columns')
y = df_test_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [114]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 2s 3ms/step - loss: 0.6518 - accuracy: 0.6199
Epoch 2/100
399/399 [==============================] - 1s 3ms/step - loss: 0.6025 - accuracy: 0.6862
Epoch 3/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5659 - accuracy: 0.7129
Epoch 4/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5409 - accuracy: 0.7346
Epoch 5/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5262 - accuracy: 0.7498
Epoch 6/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5137 - accuracy: 0.7587
Epoch 7/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5057 - accuracy: 0.7611
Epoch 8/100
399/399 [==============================] - 1s 3ms/step - loss: 0.4986 - accuracy: 0.7648
Epoch 9/100
399/399 [==============================] - 1s 3ms/step - loss: 0.4938 - accuracy: 0.7642
Epoch 10/100
399/399 [==============================] - 1s 3ms/step - loss: 0.4896 - accura

f1-score for minority class 1 improved from 0.58 to 0.79.


### Method3: SMOTE

To install imbalanced-learn library use pip install imbalanced-learn command

In [115]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [117]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [118]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [119]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 2s 5ms/step - loss: 0.6674 - accuracy: 0.5907
Epoch 2/100
399/399 [==============================] - 2s 6ms/step - loss: 0.6233 - accuracy: 0.6490
Epoch 3/100
399/399 [==============================] - 2s 5ms/step - loss: 0.5907 - accuracy: 0.6897
Epoch 4/100
399/399 [==============================] - 2s 5ms/step - loss: 0.5737 - accuracy: 0.7018
Epoch 5/100
399/399 [==============================] - 1s 4ms/step - loss: 0.5617 - accuracy: 0.7146
Epoch 6/100
399/399 [==============================] - 1s 4ms/step - loss: 0.5434 - accuracy: 0.7312
Epoch 7/100
399/399 [==============================] - 2s 4ms/step - loss: 0.5236 - accuracy: 0.7433
Epoch 8/100
399/399 [==============================] - 2s 4ms/step - loss: 0.5070 - accuracy: 0.7513
Epoch 9/100
399/399 [==============================] - 1s 4ms/step - loss: 0.4933 - accuracy: 0.7608
Epoch 10/100
399/399 [==============================] - 2s 4ms/step - loss: 0.4839 - accura

SMOT Oversampling increases f1 score of minority class 1 from 0.58 to 0.81.


### Method4: Use of Ensemble with undersampling

In [120]:
df1.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [121]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [122]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [123]:
df2 = X_train.copy()
df2['Exited'] = y_train

In [124]:
df2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
5710,0.856,0.216216,0.5,0.554265,0.333333,0,0,0.339721,1,0,0,0,1,0
3745,0.852,0.256757,0.1,0.371163,0.333333,1,1,0.980432,0,1,0,1,0,0
5429,0.664,0.405405,0.7,0.000000,0.333333,1,0,0.325318,1,0,0,1,0,0
551,0.648,0.391892,0.6,0.426077,0.000000,1,1,0.010339,0,1,0,0,1,1
8967,0.970,0.094595,0.7,0.000000,0.333333,1,1,0.417230,1,0,0,0,1,0


In [125]:
df2_class0 = df2[df2.Exited == 0]
df2_class1 = df2[df2.Exited == 1]

In [126]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis='columns')
    y_train = df_train.Exited
    return X_train, y_train

In [127]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 3ms/step - loss: 0.6645 - accuracy: 0.5974
Epoch 2/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6418 - accuracy: 0.6336
Epoch 3/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6299 - accuracy: 0.6586
Epoch 4/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6210 - accuracy: 0.6672
Epoch 5/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6130 - accuracy: 0.6806
Epoch 6/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6048 - accuracy: 0.6848
Epoch 7/100
98/98 [==============================] - 0s 3ms/step - loss: 0.5971 - accuracy: 0.6954
Epoch 8/100
98/98 [==============================] - 0s 3ms/step - loss: 0.5902 - accuracy: 0.7050
Epoch 9/100
98/98 [==============================] - 0s 3ms/step - loss: 0.5834 - accuracy: 0.7027
Epoch 10/100
98/98 [==============================] - 0s 3ms/step - loss: 0.5763 - accuracy: 0.7136
Epoch 11/

In [128]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 3ms/step - loss: 0.6614 - accuracy: 0.6131
Epoch 2/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6419 - accuracy: 0.6314
Epoch 3/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6330 - accuracy: 0.6400
Epoch 4/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6263 - accuracy: 0.6557
Epoch 5/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6175 - accuracy: 0.6669
Epoch 6/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6073 - accuracy: 0.6720
Epoch 7/100
98/98 [==============================] - 0s 3ms/step - loss: 0.5967 - accuracy: 0.6870
Epoch 8/100
98/98 [==============================] - 0s 4ms/step - loss: 0.5880 - accuracy: 0.6941
Epoch 9/100
98/98 [==============================] - 0s 3ms/step - loss: 0.5796 - accuracy: 0.6970
Epoch 10/100
98/98 [==============================] - 0s 3ms/step - loss: 0.5712 - accuracy: 0.7002
Epoch 11/

In [129]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
87/87 [==============================] - 1s 3ms/step - loss: 0.6440 - accuracy: 0.5989
Epoch 2/100
87/87 [==============================] - 0s 3ms/step - loss: 0.6193 - accuracy: 0.6433
Epoch 3/100
87/87 [==============================] - 0s 3ms/step - loss: 0.6015 - accuracy: 0.6780
Epoch 4/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5924 - accuracy: 0.6827
Epoch 5/100
87/87 [==============================] - 0s 4ms/step - loss: 0.5841 - accuracy: 0.6913
Epoch 6/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5765 - accuracy: 0.6957
Epoch 7/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5701 - accuracy: 0.7036
Epoch 8/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5663 - accuracy: 0.7051
Epoch 9/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5588 - accuracy: 0.7141
Epoch 10/100
87/87 [==============================] - 0s 3ms/step - loss: 0.5539 - accuracy: 0.7213
Epoch 11/

In [130]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [131]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.93      0.74      0.83      1593
           1       0.44      0.79      0.57       407

    accuracy                           0.75      2000
   macro avg       0.69      0.77      0.70      2000
weighted avg       0.83      0.75      0.78      2000



f1-score for minority class 1 is 0.57